In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_validate
import tqdm
import random
import seaborn as sns

from common import generate_multi_signal, get_data, get_freq_data, signal_cyclic_shift

In [19]:
X_origin, y_origin = get_data('./data/na62_11_pulses.txt', is_one_signal=False)
X_one_signal, y_one_signal = get_data('./data/na62_11_pulses.txt', is_one_signal=True)
ARGMIN_DISTR = np.argmin(X_one_signal, axis=1)



def prepare_data(X_origin, y_origin, tau_range, alpha_range, data_size=1000, to_print=False,\
                                                                         start_point=70, end_point=950):
    pos_size = int(data_size/2)
    neg_size = data_size - pos_size
    
    X = []
    y = []
    for i in range(data_size):
        alpha = random.choice(alpha_range)
        tau = random.choice(tau_range)
        
        if i < pos_size:
#             X.append(random.choice(X_one_signal))
            X.append(generate_multi_signal(X_origin, y_origin, tau, alpha,\
                                           argmin_distr=ARGMIN_DISTR)['first_impulse'][start_point:end_point])

            y.append(1)
        else:
            X.append(generate_multi_signal(X_origin, y_origin, tau, alpha,\
                                           argmin_distr=ARGMIN_DISTR)['multi_impulse'][start_point:end_point])
            y.append(0)
   
    X = np.array(X)
    y = np.array(y)
   
    if to_print:
#         print("X positive shape:", X_positive.shape)
#         print("y positive shape:", y_positive.shape)
#         print("X negative shape:", X_negative.shape)
#         print("y negative shape:", y_negative.shape)
        print("X shape:", X.shape)
        print("y shape:", y.shape)
    
    X, y = shuffle(X, y)

    return X, y

In [20]:
class Spacal_data(Dataset):
    def __init__(self, tX, tY = None, train=True):         
        if train == True:
            self.X = torch.Tensor(tX).type(torch.FloatTensor)
            self.y = torch.Tensor(tY).type(torch.FloatTensor)
            self.train = True
            
        else:
            self.X = torch.Tensor(tX).type(torch.FloatTensor)
            self.y = None
            self.train = False
            
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        #preprocessing on demand trades computations for memory
        if self.train == True:
            return self.X[idx], self.y[idx] 
        else:
            return self.X[idx]

In [47]:
alpha_range = np.array([np.around(10**i, decimals=4) for i in np.arange(-3, 3.1, 0.1)])
# alpha_range = np.array([2])
tau_range = np.arange(-25, 25, 1)
# alpha_range = np.arange(1, 10, 1)

X, y = prepare_data(X_origin, y_origin,tau_range, alpha_range, data_size=5000, to_print=True)

X shape: (5000, 880)
y shape: (5000,)


In [48]:
TEST_SIZE = len(X)//10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)

In [49]:
train_dataset = Spacal_data(X_train, y_train)
test_dataset = Spacal_data(X_test, y_test)

In [50]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(880, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 1)

    def forward(self, x):
#         print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = self.fc4(x)
#         x = x.view(2, -1)
#         print(x.shape)
        return F.sigmoid(x)  

In [51]:
def train(model, device, train_loader, optimizer, epoch, log_interval=1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
#         print(output)
#         print(target)
        criterion = nn.BCELoss()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [52]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            criterion = nn.BCELoss()
            test_loss += criterion(output, target).item() # sum up batch loss
#             print(output)
            pred = torch.round(output) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [53]:
torch.round(torch.Tensor([0.49]))

tensor([0.])

In [57]:
torch.optim.Adam

torch.optim.adam.Adam

In [62]:
BATCH_SIZE = 1024
TEST_BATCH_SIZE = 500
LR = 0.001
TO_USE_MOMENTUM = 1
EPOCHS_NUM = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True)


model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# optimizer = optim.SGD(model.parameters(), lr=LR, momentum=TO_USE_MOMENTUM)


for epoch in range(1, EPOCHS_NUM + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval=1)
    test(model, device, test_loader)

/opt/conda/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/opt/conda/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/opt/conda/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([1024, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/opt/conda/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([404])) that is different to the input size (torch.Size([404, 1])) is deprecated. Pleas

Train Epoch: 1 [0/4500 (0%)]	Loss: 0.693217
Train Epoch: 1 [1024/4500 (20%)]	Loss: 0.693206
Train Epoch: 1 [2048/4500 (40%)]	Loss: 0.693252
Train Epoch: 1 [3072/4500 (60%)]	Loss: 0.693053
Train Epoch: 1 [1616/4500 (80%)]	Loss: 0.693803

Test set: Average loss: 0.0014, Accuracy: 246/500 (49%)

Train Epoch: 2 [0/4500 (0%)]	Loss: 0.692969
Train Epoch: 2 [1024/4500 (20%)]	Loss: 0.693272
Train Epoch: 2 [2048/4500 (40%)]	Loss: 0.693107
Train Epoch: 2 [3072/4500 (60%)]	Loss: 0.693164
Train Epoch: 2 [1616/4500 (80%)]	Loss: 0.693190

Test set: Average loss: 0.0014, Accuracy: 246/500 (49%)

Train Epoch: 3 [0/4500 (0%)]	Loss: 0.693180
Train Epoch: 3 [1024/4500 (20%)]	Loss: 0.693166
Train Epoch: 3 [2048/4500 (40%)]	Loss: 0.693164
Train Epoch: 3 [3072/4500 (60%)]	Loss: 0.693167
Train Epoch: 3 [1616/4500 (80%)]	Loss: 0.693079

Test set: Average loss: 0.0014, Accuracy: 254/500 (51%)

Train Epoch: 4 [0/4500 (0%)]	Loss: 0.693086
Train Epoch: 4 [1024/4500 (20%)]	Loss: 0.693212
Train Epoch: 4 [2048/4500 

Train Epoch: 31 [3072/4500 (60%)]	Loss: 0.692771
Train Epoch: 31 [1616/4500 (80%)]	Loss: 0.693221

Test set: Average loss: 0.0014, Accuracy: 246/500 (49%)

Train Epoch: 32 [0/4500 (0%)]	Loss: 0.692608
Train Epoch: 32 [1024/4500 (20%)]	Loss: 0.693321
Train Epoch: 32 [2048/4500 (40%)]	Loss: 0.692994
Train Epoch: 32 [3072/4500 (60%)]	Loss: 0.692891
Train Epoch: 32 [1616/4500 (80%)]	Loss: 0.693056

Test set: Average loss: 0.0014, Accuracy: 245/500 (49%)

Train Epoch: 33 [0/4500 (0%)]	Loss: 0.693019
Train Epoch: 33 [1024/4500 (20%)]	Loss: 0.693090
Train Epoch: 33 [2048/4500 (40%)]	Loss: 0.692736
Train Epoch: 33 [3072/4500 (60%)]	Loss: 0.692926
Train Epoch: 33 [1616/4500 (80%)]	Loss: 0.692945

Test set: Average loss: 0.0014, Accuracy: 263/500 (53%)

Train Epoch: 34 [0/4500 (0%)]	Loss: 0.692929
Train Epoch: 34 [1024/4500 (20%)]	Loss: 0.692908
Train Epoch: 34 [2048/4500 (40%)]	Loss: 0.692870
Train Epoch: 34 [3072/4500 (60%)]	Loss: 0.693122
Train Epoch: 34 [1616/4500 (80%)]	Loss: 0.692897

Test

Train Epoch: 59 [0/4500 (0%)]	Loss: 0.692178
Train Epoch: 59 [1024/4500 (20%)]	Loss: 0.692754
Train Epoch: 59 [2048/4500 (40%)]	Loss: 0.692443
Train Epoch: 59 [3072/4500 (60%)]	Loss: 0.692788
Train Epoch: 59 [1616/4500 (80%)]	Loss: 0.692776

Test set: Average loss: 0.0014, Accuracy: 262/500 (52%)

Train Epoch: 60 [0/4500 (0%)]	Loss: 0.692315
Train Epoch: 60 [1024/4500 (20%)]	Loss: 0.692887
Train Epoch: 60 [2048/4500 (40%)]	Loss: 0.692826
Train Epoch: 60 [3072/4500 (60%)]	Loss: 0.691987
Train Epoch: 60 [1616/4500 (80%)]	Loss: 0.692584

Test set: Average loss: 0.0014, Accuracy: 260/500 (52%)

Train Epoch: 61 [0/4500 (0%)]	Loss: 0.692806
Train Epoch: 61 [1024/4500 (20%)]	Loss: 0.692694
Train Epoch: 61 [2048/4500 (40%)]	Loss: 0.692335
Train Epoch: 61 [3072/4500 (60%)]	Loss: 0.692378
Train Epoch: 61 [1616/4500 (80%)]	Loss: 0.691302

Test set: Average loss: 0.0014, Accuracy: 248/500 (50%)

Train Epoch: 62 [0/4500 (0%)]	Loss: 0.692915
Train Epoch: 62 [1024/4500 (20%)]	Loss: 0.692364
Train Epo

Train Epoch: 89 [1024/4500 (20%)]	Loss: 0.690891
Train Epoch: 89 [2048/4500 (40%)]	Loss: 0.692230
Train Epoch: 89 [3072/4500 (60%)]	Loss: 0.691761
Train Epoch: 89 [1616/4500 (80%)]	Loss: 0.690214

Test set: Average loss: 0.0014, Accuracy: 253/500 (51%)

Train Epoch: 90 [0/4500 (0%)]	Loss: 0.689671
Train Epoch: 90 [1024/4500 (20%)]	Loss: 0.692589
Train Epoch: 90 [2048/4500 (40%)]	Loss: 0.690974
Train Epoch: 90 [3072/4500 (60%)]	Loss: 0.692223
Train Epoch: 90 [1616/4500 (80%)]	Loss: 0.692352

Test set: Average loss: 0.0014, Accuracy: 259/500 (52%)

Train Epoch: 91 [0/4500 (0%)]	Loss: 0.691679
Train Epoch: 91 [1024/4500 (20%)]	Loss: 0.691919
Train Epoch: 91 [2048/4500 (40%)]	Loss: 0.692106
Train Epoch: 91 [3072/4500 (60%)]	Loss: 0.689878
Train Epoch: 91 [1616/4500 (80%)]	Loss: 0.690691

Test set: Average loss: 0.0014, Accuracy: 255/500 (51%)

Train Epoch: 92 [0/4500 (0%)]	Loss: 0.692910
Train Epoch: 92 [1024/4500 (20%)]	Loss: 0.689693
Train Epoch: 92 [2048/4500 (40%)]	Loss: 0.692420
Train